# Clone Repo

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/ahmedoy/trigs

Cloning into 'trigs'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 57 (delta 17), reused 44 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 36.17 KiB | 9.04 MiB/s, done.
Resolving deltas: 100% (17/17), done.


# Download Data

In [4]:
import gdown
import zipfile
import os
base_url = 'https://drive.google.com/uc?id='
clean_train_cifar = base_url + '1VbctgwsMfMLFJwSLeVqykKnoBQ7Mvi_B'
poison_train_cifar = base_url + '1vhhsUR6HPttYvLgTWN1_zE2RfI6BXvok'
save_dir = '/content/drive/MyDrive/datasets/cifar/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Step 2: Download the datasets to Google Drive
poison_train_zip = save_dir + 'poison_train_cifar.zip'
clean_train_zip = save_dir + 'clean_train_cifar.zip'
# Download the zip files
gdown.download(poison_train_cifar, poison_train_zip, quiet=False)
gdown.download(clean_train_cifar, clean_train_zip, quiet=False)

# Step 3: Unzip the datasets in Google Drive
with zipfile.ZipFile(poison_train_zip, 'r') as zip_ref:
    zip_ref.extractall(save_dir + 'poison_train_cifar/')

with zipfile.ZipFile(clean_train_zip, 'r') as zip_ref:
    zip_ref.extractall(save_dir + 'clean_train_cifar/')

print("Datasets unzipped and saved in Google Drive.")


Downloading...
From (original): https://drive.google.com/uc?id=1vhhsUR6HPttYvLgTWN1_zE2RfI6BXvok
From (redirected): https://drive.google.com/uc?id=1vhhsUR6HPttYvLgTWN1_zE2RfI6BXvok&confirm=t&uuid=945d12fa-19b7-45e5-a4f9-818d66d11963
To: /content/drive/MyDrive/datasets/cifar/poison_train_cifar.zip
100%|██████████| 1.84G/1.84G [00:49<00:00, 37.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1VbctgwsMfMLFJwSLeVqykKnoBQ7Mvi_B
From (redirected): https://drive.google.com/uc?id=1VbctgwsMfMLFJwSLeVqykKnoBQ7Mvi_B&confirm=t&uuid=5d47ea26-6ca7-46bf-a496-6febcab392dc
To: /content/drive/MyDrive/datasets/cifar/clean_train_cifar.zip
100%|██████████| 1.86G/1.86G [00:38<00:00, 48.6MB/s]


Datasets unzipped and saved in Google Drive.


# Setup Library Versions

Not required so far. Colab's libraries seem to work by default.

# Check Help Method in the code

In [ ]:
!PYTHONPATH=/content/trigs python /content/trigs/trigs/generate_model_signature.py -h


usage: generate_model_signature.py [-h] --dataset_name DATASET_NAME [--model_name MODEL_NAME]
                                   [--weights_path WEIGHTS_PATH] [--iterations ITERATIONS]
                                   --learning_rate LEARNING_RATE --output_dir OUTPUT_DIR
                                   [--opt_type {ADAM,SGD}] [--blur_freq BLUR_FREQ]
                                   [--blur_sigma BLUR_SIGMA] [--blur_hks BLUR_HKS]
                                   [--clipping_val CLIPPING_VAL] [--normalize_grad]
                                   [--loss_type {logit,ce}] [--standardize_output]
                                   [--clamp_pixels_freq CLAMP_PIXELS_FREQ] [--lambda_tv LAMBDA_TV]
                                   [--batch_size BATCH_SIZE] [--debug]

options:
  -h, --help            show this help message and exit
  --dataset_name DATASET_NAME
                        Name of dataset used to train the probe models. Use Cifar10Ext for CIFAR10
                        and 

# Get Clean and Poisoned Signatures


Note that we have modified the original TRIGS repo to make this work



In [19]:
import subprocess
import os

# Function to create a placeholder weights file if it does not exist
def create_placeholder_file(file_path):
    with open(file_path, 'w') as f:
        f.write("")  # Create an empty file

for i in range(2):  # Loop from 0 to 499
    num_str = f"{i:04d}"  # Format the number to four digits

    output_dir = f"/content/drive/MyDrive/datasets/cifar/signatures/clean/{num_str}"
    weights_path = f"/content/drive/MyDrive/datasets/cifar/clean_train_cifar/clean_vggmod_CIFAR-10_{num_str}.pt"

    # Check if the weights file exists
    if not os.path.exists(weights_path):
        print(f"Creating placeholder weights file: {weights_path}")
        create_placeholder_file(weights_path)

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the command
    command = [
        "python3", "/content/trigs/trigs/generate_model_signature.py",
        "--dataset_name", "Cifar10Ext",
        "--model_name", "vggmod",
        "--weights_path", weights_path,
        "--iterations", "200",
        "--learning_rate", "10",
        "--output_dir", output_dir,
        "--opt_type", "ADAM",
        "--lambda_tv", "1e-3",
        "--batch_size", "250"
    ]

    # Execute the command and capture output
    try:
        result = subprocess.run(command, env={"PYTHONPATH": "/content/trigs"}, check=True, text=True, capture_output=True)
        print(f"Command executed successfully for {num_str}: {result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred for {num_str}: {e.stderr}")


Command executed successfully for 0000: SUCCESS!

Command executed successfully for 0001: SUCCESS!



In [21]:
for i in range(2):  # Loop from 0 to 499
    num_str = f"{i:04d}"  # Format the number to four digits

    output_dir = f"/content/drive/MyDrive/datasets/cifar/signatures/poisoned/{num_str}"
    weights_path = f"/content/drive/MyDrive/datasets/cifar/poison_train_cifar/poisoned_vggmod_CIFAR-10_{num_str}.pt"

    # Check if the weights file exists
    if not os.path.exists(weights_path):
        print(f"Creating placeholder weights file: {weights_path}")
        create_placeholder_file(weights_path)

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the command
    command = [
        "python3", "/content/trigs/trigs/generate_model_signature.py",
        "--dataset_name", "Cifar10Ext",
        "--model_name", "vggmod",
        "--weights_path", weights_path,
        "--iterations", "200",
        "--learning_rate", "10",
        "--output_dir", output_dir,
        "--opt_type", "ADAM",
        "--lambda_tv", "1e-3",
        "--batch_size", "250"
    ]

    # Execute the command and capture output
    try:
        result = subprocess.run(command, env={"PYTHONPATH": "/content/trigs"}, check=True, text=True, capture_output=True)
        print(f"Command executed successfully for {num_str}: {result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred for {num_str}: {e.stderr}")

Command executed successfully for 0000: SUCCESS!

Command executed successfully for 0001: SUCCESS!

